[How does Julia merge several exprs to one expr?](https://discourse.julialang.org/t/how-does-julia-merge-several-exprs-to-one-expr/77498)

In [2]:
nn=[:(at=at), :(ct=ct)]
:($(nn...),)

:((at = at, ct = ct))

Thanks a lot to your help and your advice.
In broader context, my target is to creat a set of local varialbes. Firstly, create a NamedTuple came to my mind, but in practise, that is not convinient. So now I can get the target just like this:

In [3]:
macro localvars(expr)
    bb=[]
    for cc in expr.args
        if cc isa Expr && cc.head==:(=)
            append!(bb,[cc.args[1]])
        end
    end
    cvv=Expr(:tuple,(:($x=$x) for x in bb)...)
    bbres=quote
        let
            $expr
            $cvv
        end
    end |> esc
end

@localvars (macro with 1 method)

In [4]:
vars=@localvars begin
    a=1
    b=2
    c=a+b
end 

(a = 1, b = 2, c = 3)

In [5]:
vars.a |> display
vars.c |> display

1

3

You might want to take a look at StaticModules.jl; maybe it can cover some of what you need:

In [6]:
using StaticModules
@staticmodule vars begin
    a=1
    b=2
    c=a+b
end

StaticModule vars containing
  a = 1
  b = 2
  c = 3

In [8]:
vars.c

3

[Can I concatenate expressions in macro?](https://discourse.julialang.org/t/can-i-concatenate-expressions-in-macro/69220)

i have array of expressions generated in macro and i want to return it all so they will be executed one by one

You could look at how a block of expressions is normally represented and try to rebuild that. For example

In [15]:
a = quote
        a = 1+1
        b = a + 4
    end


quote
    #= e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb:2 =#
    a = 1 + 1
    #= e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb:3 =#
    b = a + 4
end

In [16]:
dump(a)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol a
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Int64 1
            3: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol b
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Symbol a
            3: Int64 4


Here the LineNumberNode is not interesting, but what we see is that we just have an expression with head being :block, and the remaining expressions are in the args. So if you have an array with expressions you could probably do something along the lines of

此处 LineNumberNode 并不有趣，但我们看到只有一个表达式，其头部为 :block，其余表达式在参数中。因此，如果您有一个包含表达式的数组，则可能可以执行类似以下操作
```julia
macro mymacro(args...)
    exprs = preprocess(args) # Whatever you do to create the expression list
    Expr(:block, exprs...) # Create a block expression with all sub-expressions
end
```

Just one more thing before and after this list of expression i want to put the while and end
so I want to add this while and end as expression - yet I get the error in both cases - so i suppose I need somehow escape while and end phrases but how to do this?

push!(tmp, quote while($offsetIter <32) end)

push…

push!(tmp, quote end end)

So what you want the macro to generate in the end is something like this?

```julia
while condition
    expr1
    expr2
    ...
end
```

In [19]:
# So what you want the macro to generate in the end is something like this?
a = quote
        while 3 < 4
            1+1
        end
    end

quote
    #= e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb:3 =#
    while 3 < 4
        #= e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb:4 =#
        1 + 1
        #= e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb:5 =#
    end
end

In [21]:
dump(a)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 3
      file: Symbol e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb
    2: Expr
      head: Symbol while
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol <
            2: Int64 3
            3: Int64 4
        2: Expr
          head: Symbol block
          args: Array{Any}((3,))
            1: LineNumberNode
              line: Int64 4
              file: Symbol e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol +
                2: Int64 1
                3: Int64 1
            3: LineNumberNode
              line: Int64 5
              file: Symbol e:\Projects.jl\Training3.jl\macro4_StaticModules\merge exprs to one expr.ipynb


In [22]:
exprs = [:(a = 1), :(b = a + 1), :(a + b)]
Expr(:while, Expr(:call, :<, :offsetIter, 32), Expr(:block, exprs...))

:(while offsetIter < 32
      a = 1
      b = a + 1
      a + b
  end)

In [13]:
let v = [:(a = 1), :(b = a + 1), :(a + b)]
    Expr(:block, v...)
end

quote
    a = 1
    b = a + 1
    a + b
end